# Imports 

In [1]:
import numpy as np
import pandas as pd

# Get the data

In [2]:
%%time

CHOSEN_SHEETS_SUMMARY = ["England", "Scotland", "Wales", "Northern Ireland"]
CHOSEN_SHEETS_UNIS = ["Q01", "Q02", "Q03", "Q04", "Q05", "Q06", "Q07", "Q08", "Q09", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15",
                     "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"]
CHOSEN_SHEETS_COURSE = ["NSS1", "NSS2", "NSS3"]

summary_data_1819 = pd.read_excel(r"data/summary_data_2018_2019.xls", skiprows = 5, nrows = 37, sheet_name = CHOSEN_SHEETS_SUMMARY) #skip nhs placement questions
summary_data_2122 = pd.read_excel(r"data/summary_data_2021_2022.xls", skiprows = 5, nrows = 37, sheet_name = CHOSEN_SHEETS_SUMMARY) #skip nhs placement questions

unis_data_1819 = pd.read_excel(r"data/unis_2018_2019.xls", skiprows = 5, sheet_name = CHOSEN_SHEETS_UNIS, header = None)
unis_data_2122 = pd.read_excel(r"data/unis_2021_2022.xls", skiprows = 5, sheet_name = CHOSEN_SHEETS_UNIS, header = None)

course_data_18 = pd.read_excel(r"data/course_data_2018.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_19 = pd.read_excel(r"data/course_data_2019.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_21 = pd.read_excel(r"data/course_data_2021.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_22 = pd.read_excel(r"data/course_data_2022.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)

Wall time: 46.8 s


# Cleaning

#### 1. Cleaning summary data

In [3]:
def weighted_avg(file, idx, left_side):
    if left_side:
        return (file.iloc[idx][1] * file.iloc[idx][2] + file.iloc[idx][3] * file.iloc[idx][4]) / (file.iloc[idx][2]+ file.iloc[idx][4])
    return (file.iloc[idx][5] * file.iloc[idx][6] + file.iloc[idx][7] * file.iloc[idx][8]) / (file.iloc[idx][6]+ file.iloc[idx][8])
    
def clean(file, sheet_name, left_side):
    #optimise so that first columns are dropped after usage so left_side not needed, and copy data so can be run multiple times
    df = file.get(sheet_name).copy()
    df.dropna(axis = "rows", inplace = True) #drop empty rows - just text
    df.reset_index(inplace = True, drop = True) #reset index

    average_pct = []
    response_counts = []
    for i in range(len(df)):
        average_pct.append(weighted_avg(df, i, left_side))
        if left_side:
            response_counts.append(df.iloc[i][2] + df.iloc[i][4])
        else:
            response_counts.append(df.iloc[i][6] + df.iloc[i][8]) 
        
    df.drop(df.columns[[1,2,3,4,5,6,7,8]], axis=1, inplace=True) #drop columns, no longer needed
    df["score"] = average_pct #add score as feature
    
    df = df.T #set transpose as we want Qs af features
    df.rename(columns= lambda x: "Q"+str(x+1), inplace = True) #rename
    df.drop("Unnamed: 0", inplace = True, axis = "rows") #drop question row, now in header
    df.rename(index = {"score": sheet_name}, inplace = True) #keep track of rows
    df["avg_res"] = sum(response_counts)/df.shape[1] #add respsonse count
    
    return df

In [4]:
df_18 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_1819, sheet, True)
    df_18 = pd.concat([df_18, temp_df])
    
df_21 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_2122, sheet, True)
    df_21 = pd.concat([df_21, temp_df])

df_19 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_1819, sheet, False)
    df_19 = pd.concat([df_19, temp_df])
    
df_22 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_2122, sheet, False)
    df_22 = pd.concat([df_22, temp_df])
    
#take average of two years
summary_1819 = (df_18 + df_19) / 2
summary_2122 = (df_21 + df_22) / 2

#add total row by getting weighted avg and sum of repsonses
values_1819 = []
for i in range(summary_1819.shape[1]-1):
    values_1819.append(sum(summary_1819.iloc[:,i] * summary_1819.iloc[:,-1]) / sum(summary_1819.iloc[:,-1]))
values_1819.append(sum(summary_1819.iloc[:,-1]))

values_2122 = []
for i in range(summary_2122.shape[1]-1):
    values_2122.append(sum(summary_2122.iloc[:,i] * summary_2122.iloc[:,-1]) / sum(summary_2122.iloc[:,-1]))
values_2122.append(sum(summary_2122.iloc[:,-1]))

summary_1819.loc["UK"] = np.array(values_1819)
summary_2122.loc["UK"] = np.array(values_2122)

#### 2. Cleaning unis data

In [5]:
def clean_unis(file, sheet_name):
    df = file.get(sheet_name).copy()
    df.drop([0,3,4,5,6,7,9,10,11,12,13,14], inplace = True, axis = "columns")
    df.dropna(inplace = True, axis = "rows")
    df.reset_index(inplace = True, drop = True) #reset row index
    df[sheet_name] = (df.iloc[:,1] + df.iloc[:,2]) / 2 #add average result of the two years
    df.rename(columns = {1:'University'}, inplace = True) #rename column
    df.drop([2,8], inplace = True, axis = "columns")
    
    return df

In [6]:
for sheet in CHOSEN_SHEETS_UNIS:
    #create df for q1, keep adding scores only afterwards, uni names are returned for clarity
    if sheet == "Q01":
        unis_1819 = clean_unis(unis_data_1819, sheet)
    else:
        unis_1819[sheet] = clean_unis(unis_data_1819, sheet).iloc[:,1]
        
for sheet in CHOSEN_SHEETS_UNIS:
    #create df for q1, keep adding scores only afterwards, uni names are returned for clarity
    if sheet == "Q01":
        unis_2122 = clean_unis(unis_data_2122, sheet)
    else:
        unis_2122[sheet] = clean_unis(unis_data_2122, sheet).iloc[:,1]

#### 3. Cleaning course data

In [18]:
def clean_courses(file, sheet, question_numbers, new_data):
    df = file.get(sheet).copy()
    #21-22 files do not include last column
    if new_data:
        df.drop([0,1,2,4,6,7,8,9,10,11,12,14,15,16], inplace = True, axis = "columns") #drop not needed stuff
    else:
        df.drop([0,1,2,4,6,7,8,9,10,11,12,14,15,16,17], inplace = True, axis = "columns") #drop not needed stuff
    df.rename(columns = {3: "course", 5:"question", 13: "result"}, inplace = True) #rename so I actually know what is what
    
    #setup df
    course_data = pd.DataFrame()
    course_data["course"] = df.groupby(by = ["course"], as_index = False).mean(numeric_only = True)["course"]
    
    #get average by question and course
    for q in question_numbers:
        temp = df[df["question"] == q].copy()
        course_data[q] = temp.groupby(by = ["course"], as_index = False).mean(numeric_only = True)["result"]
        
    return course_data

In [177]:
course_18 = clean_courses(course_data_18, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_19 = clean_courses(course_data_19, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_21 = clean_courses(course_data_21, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)
course_22 = clean_courses(course_data_22, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)

In [178]:
#save media
media_18 = course_18.iloc[5]
media_19 = course_19.iloc[14]
media_21 = course_21.iloc[14]
media_22 = course_22.iloc[14]

#remove Humanities and liberal arts (non-specific), doesnt exist in other years
#also remove comms and media because it needs to be moved terrible code, but it is late
course_18.drop([5,12], axis = "rows", inplace = True) 
course_18.reset_index(drop = True, inplace = True)
course_19.drop([14], axis = "rows", inplace = True) 
course_19.reset_index(drop = True, inplace = True)
course_21.drop([14], axis = "rows", inplace = True) 
course_21.reset_index(drop = True, inplace = True)
course_22.drop([14], axis = "rows", inplace = True) 
course_22.reset_index(drop = True, inplace = True)

#readding media and comms
course_18 = course_18.append(media_18, ignore_index = True)
course_19 = course_19.append(media_19, ignore_index = True)
course_21 = course_21.append(media_21, ignore_index = True)
course_22 = course_22.append(media_22, ignore_index = True)

C:\Users\Áron\AppData\Local\Temp\ipykernel_5928\2837721766.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_18 = course_18.append(media_18, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_5928\2837721766.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_19 = course_19.append(media_19, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_5928\2837721766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_21 = course_21.append(media_21, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_5928\2837721766.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_22 = cour

In [180]:
course_18

,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,"Agriculture, food and related studies",0.887284,0.826351,0.816219,0.776903,0.796525,0.838622,0.766719,0.737115,0.745991,...,0.766332,0.820556,0.821204,0.699568,0.854299,0.805291,0.747590,0.573751,0.499341,0.787574
1,"Architecture, building and planning",0.866149,0.810334,0.834065,0.805786,0.814275,0.849751,0.807264,0.702341,0.727188,...,0.771932,0.852341,0.827366,0.715921,0.875445,0.808549,0.743875,0.594312,0.547516,0.794308
2,Biological and sport sciences,0.900285,0.836669,0.850092,0.812972,0.840142,0.852050,0.816582,0.744191,0.755934,...,0.798417,0.816391,0.842084,0.723049,0.881174,0.835966,0.777120,0.631830,0.593156,0.837787
3,Business and management,0.874973,0.796238,0.796573,0.792341,0.821510,0.855017,0.799959,0.759801,0.760513,...,0.799049,0.824351,0.833410,0.692107,0.855981,0.823097,0.758211,0.630515,0.593852,0.813368
4,Combined and general studies,0.899513,0.839913,0.908175,0.820987,0.869762,0.864800,0.743788,0.758825,0.802063,...,0.831438,0.866737,0.885112,0.636688,0.737062,0.840900,0.775250,0.573625,0.570788,0.892937
5,Computing,0.845718,0.769177,0.810913,0.782875,0.776605,0.775730,0.815188,0.711088,0.779684,...,0.778428,0.740065,0.826512,0.679813,0.864679,0.819393,0.743501,0.594845,0.536805,0.779561
6,Creative arts and design,0.875059,0.854587,0.817700,0.805982,0.839547,0.839464,0.838231,0.716921,0.738579,...,0.741874,0.799074,0.805527,0.750674,0.844091,0.814005,0.767428,0.628925,0.563463,0.795751
7,Education and teaching,0.901990,0.856634,0.860051,0.859054,0.893320,0.897605,0.887557,0.806754,0.795467,...,0.775887,0.829654,0.810341,0.737525,0.881510,0.854799,0.816146,0.655060,0.594161,0.855477
8,Engineering and technology,0.819575,0.734890,0.821945,0.796399,0.761052,0.794931,0.746095,0.683187,0.737130,...,0.788906,0.793720,0.811007,0.673140,0.857451,0.793968,0.704305,0.560725,0.515016,0.758365
9,Geographical and environmental studies,0.920470,0.871597,0.865138,0.803969,0.889603,0.881918,0.798925,0.756980,0.766880,...,0.849575,0.865703,0.886767,0.727030,0.885674,0.865111,0.783539,0.597810,0.524090,0.870777


In [181]:
course_19

,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,"Agriculture, food and related studies",0.913953,0.861290,0.832329,0.807849,0.839714,0.852082,0.793657,0.780600,0.786618,...,0.809922,0.836867,0.855214,0.717780,0.864806,0.849357,0.796000,0.642129,0.544129,0.826180
1,"Architecture, building and planning",0.862965,0.827728,0.855180,0.821112,0.826238,0.873078,0.812631,0.696077,0.718857,...,0.775797,0.850562,0.849097,0.735598,0.878842,0.835211,0.767195,0.630345,0.575106,0.807283
2,Biological and sport sciences,0.910678,0.839410,0.851645,0.828418,0.843139,0.865514,0.820830,0.768602,0.765160,...,0.807614,0.837399,0.859321,0.724781,0.873856,0.858370,0.805242,0.665133,0.595343,0.845998
3,Business and management,0.888615,0.819659,0.817491,0.822787,0.834188,0.874665,0.812460,0.779021,0.780723,...,0.799479,0.838411,0.836211,0.706141,0.853949,0.842354,0.781882,0.660752,0.597993,0.828047
4,Combined and general studies,0.940827,0.848809,0.880482,0.769409,0.844355,0.873964,0.748473,0.738364,0.742618,...,0.861736,0.903564,0.881645,0.639209,0.771373,0.838818,0.778900,0.568455,0.550391,0.879000
5,Computing,0.850016,0.775683,0.818703,0.785088,0.788799,0.803605,0.820026,0.710777,0.767598,...,0.800480,0.737537,0.856890,0.693510,0.853988,0.841468,0.747095,0.594233,0.531827,0.779933
6,"Design, and creative and performing arts",0.883334,0.857626,0.811298,0.817653,0.846483,0.842428,0.844046,0.728970,0.750540,...,0.733142,0.771397,0.794535,0.746897,0.833210,0.816861,0.771891,0.633479,0.562822,0.789632
7,Education and teaching,0.908200,0.864962,0.863672,0.870274,0.897347,0.909561,0.886726,0.796062,0.793715,...,0.767866,0.825260,0.812201,0.728509,0.884059,0.856168,0.815637,0.650409,0.570426,0.856424
8,Engineering and technology,0.814016,0.739178,0.796666,0.764802,0.731728,0.770775,0.738287,0.660911,0.721186,...,0.773809,0.765411,0.812393,0.649360,0.855272,0.793690,0.704008,0.554109,0.483733,0.744172
9,"Geography, earth and environmental studies",0.920033,0.867076,0.874139,0.792835,0.876289,0.890883,0.820220,0.727983,0.730822,...,0.859054,0.882024,0.906320,0.740365,0.884417,0.881383,0.792509,0.594863,0.495107,0.871461


# Save and load data

In [21]:
summary_1819.to_pickle("data/summary_1819.pkl")
summary_2122.to_pickle("data/summary_2122.pkl")
unis_1819.to_pickle("data/unis_1819.pkl")
unis_2122.to_pickle("data/unis_2122.pkl")

In [22]:
pd.read_pickle("data/summary_1819.pkl")

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,avg_res
England,88.77816,81.924411,84.800519,81.225215,84.075056,84.886815,80.889026,72.525879,72.712395,74.60084,...,86.632812,86.817193,68.538242,83.659492,83.902753,75.447773,60.91548,55.902584,83.150482,274799.296296
Scotland,89.457671,83.012992,85.54163,80.55478,83.527915,84.040667,78.568853,71.681497,73.16735,65.166609,...,87.430486,87.890169,67.254589,85.835967,86.415676,74.458033,55.403336,52.364461,83.527527,24350.777778
Wales,89.935977,83.503667,84.982081,82.014255,84.407333,85.421609,82.467954,75.067815,75.181148,75.042689,...,86.807669,87.224629,71.852159,85.437866,86.321741,78.964212,64.790899,58.695526,84.920995,16079.574074
Northern Ireland,89.024086,82.205276,85.253734,83.277861,83.700776,85.20134,84.200163,73.878637,74.954569,68.803944,...,88.12664,88.126266,68.900289,88.274402,83.822866,74.552509,59.371378,57.498343,84.522136,7774.462963
UK,88.892944,82.091855,84.876337,81.263357,84.04134,84.857219,80.87241,72.621323,72.923558,73.772077,...,86.737608,86.949874,68.615155,84.02318,84.210696,75.526663,60.655686,55.813295,83.30006,323004.111111


In [23]:
pd.read_pickle("data/summary_2122.pkl")

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,avg_res
England,79.737075,84.147697,77.662464,81.305213,75.903553,79.071665,79.465628,81.224499,76.59048,68.78683,...,74.858694,66.321623,78.887684,68.548402,51.533327,52.771014,52.771014,75.427824,75.427824,277392.405405
Scotland,82.109712,86.462607,80.596142,83.84219,77.596963,79.973417,81.229188,82.6085,76.144778,65.740663,...,77.515323,66.992655,84.077905,68.420797,48.336113,52.010475,52.010475,79.074508,79.074508,25740.797297
Wales,80.525789,85.011885,79.146727,81.53957,76.456929,79.132272,79.015333,81.310633,77.152465,69.522817,...,75.244072,68.246707,80.894096,69.954157,53.914158,55.46916,55.46916,76.422458,76.422458,15153.256757
Northern Ireland,81.095008,85.277551,77.698817,82.520124,78.952124,81.038071,80.047217,82.278878,80.933878,67.921884,...,80.587072,67.513943,79.152724,69.315699,54.198072,56.365123,56.365123,79.418958,79.418958,7575.351351
UK,79.992741,84.397011,77.96407,81.544757,76.133924,79.191428,79.597518,81.362342,76.682378,68.560322,...,75.219638,66.491868,79.397139,68.62153,51.45343,52.919959,52.919959,75.854922,75.854922,325861.810811


In [24]:
pd.read_pickle("data/unis_1819.pkl")

,University,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,University of Aberdeen,91.345,84.160,88.010,80.060,85.455,83.625,76.190,72.820,77.140,...,84.495,90.445,90.810,72.345,87.335,90.275,78.440,58.760,44.100,86.645
1,Abertay University,90.895,84.025,82.300,80.950,83.200,85.370,78.330,77.285,76.705,...,87.470,88.060,89.715,68.865,86.465,88.630,76.885,59.520,53.505,82.965
2,Aberystwyth University,95.120,89.995,90.100,85.585,89.530,89.910,86.170,84.175,82.035,...,91.680,91.530,91.270,75.230,88.050,89.925,85.540,77.150,58.170,90.470
3,Abingdon and Witney College,94.310,92.630,91.380,87.960,84.600,88.250,82.260,78.610,82.040,...,75.910,75.335,83.585,71.545,89.340,86.545,87.170,72.340,47.770,89.215
4,ACM Guildford Limited,80.725,68.250,51.025,50.355,65.165,65.820,65.425,52.195,60.040,...,61.215,53.395,64.325,59.545,75.470,65.185,61.785,42.940,36.845,50.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,York College,88.030,82.215,77.800,77.185,73.715,76.060,78.795,71.130,79.185,...,67.935,74.605,70.590,65.855,76.210,72.150,74.320,52.810,36.535,75.000
412,York St John University,91.440,85.700,83.405,80.440,86.385,85.280,82.690,76.280,72.990,...,86.140,88.790,86.960,71.890,85.160,87.645,80.175,62.745,56.920,85.470
413,Medway School of Pharmacy,86.960,82.100,85.215,77.310,81.055,87.180,84.385,62.600,64.405,...,79.510,86.950,88.235,57.845,86.600,83.455,72.000,69.220,49.530,84.800
414,Hull and York Medical School,83.010,86.460,92.515,78.320,80.010,80.805,93.295,62.615,62.110,...,78.320,91.735,91.980,79.100,91.225,84.935,54.185,41.100,49.440,76.725


In [25]:
pd.read_pickle("data/unis_2122.pkl")

,University,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,University of Aberdeen,89.710,83.805,87.465,80.030,83.305,84.815,76.645,68.720,73.830,...,80.765,83.885,84.755,65.650,78.815,90.235,73.165,53.870,52.595,85.085
1,Abertay University,89.885,82.375,82.535,77.345,82.740,81.375,78.440,74.410,72.840,...,78.040,77.545,79.580,55.545,77.540,84.605,69.930,52.170,45.015,80.000
2,Aberystwyth University,91.130,85.960,87.490,79.215,84.445,85.285,80.410,79.010,78.460,...,83.050,82.975,83.150,66.420,79.960,89.240,79.970,68.370,61.290,85.165
3,Abingdon and Witney College,84.135,83.975,75.960,72.115,79.710,86.060,83.560,66.500,77.210,...,72.885,71.155,73.890,65.865,81.060,82.980,86.500,65.415,54.420,84.500
4,ACM Guildford Limited,72.950,66.890,56.565,56.515,59.605,65.260,59.180,58.430,57.515,...,50.790,50.445,52.125,43.700,61.310,49.255,55.705,29.470,28.670,44.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,University of York,87.670,82.130,86.295,74.125,83.050,83.740,73.430,66.965,69.055,...,80.180,83.165,83.420,59.365,75.970,84.455,67.900,44.450,41.100,78.975
408,York College,84.225,88.335,83.150,86.485,88.150,86.295,84.445,66.475,71.650,...,75.715,85.930,76.280,82.960,70.555,84.225,75.930,70.555,71.130,82.595
409,York St John University,86.610,81.475,82.565,78.455,81.960,82.590,76.970,73.375,73.825,...,75.420,80.700,78.585,65.885,78.710,82.640,73.895,54.250,56.750,77.995
410,Hull and York Medical School,77.170,82.070,92.415,77.335,69.335,80.280,93.980,57.075,63.960,...,73.915,85.540,89.470,70.660,88.980,79.655,55.170,42.885,36.350,73.625
